<font size="+3"><strong>Preparing Mexico Data</strong></font>

In [1]:
import pandas as pd
import os

# Import

The first part of any data science project is preparing your data, which means making sure its in the right place and format for you to conduct your analysis. The first step of any data preparation is importing your raw data and cleaning it. 

If you look in the `small-data` directory, you'll see that the data for this project comes in three CSV files: `mexico-real-estate-1.csv`, `mexico-real-estate-2.csv`, and `mexico-real-estate-3.csv`.

**Task 1.2.1:** Read these three files into three separate DataFrames named `df1`, `df2`, and `df3`, respectively.

In [2]:
df1 = pd.read_csv("data/mexico-real-estate-1.csv")
df2 = pd.read_csv("data/mexico-real-estate-2.csv")
df3 = pd.read_csv("data/mexico-real-estate-3.csv")

**Task 1.2.2:** Inspect `df1` by looking at its **`shape`** attribute. Then use the **`info`** method to see the data types and number of missing values for each column. Finally, use the **`head`** method to determine to look at the first five rows of your dataset.

In [3]:
df1.shape

(700, 6)

In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   property_type  700 non-null    object 
 1   state          700 non-null    object 
 2   lat            583 non-null    float64
 3   lon            583 non-null    float64
 4   area_m2        700 non-null    int64  
 5   price_usd      700 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 32.9+ KB


In [5]:
df1.head()

,property_type,state,lat,lon,area_m2,price_usd
0,house,Estado de México,19.560181,-99.233528,150,"$67,965.56"
1,house,Nuevo León,25.688436,-100.198807,186,"$63,223.78"
2,apartment,Guerrero,16.767704,-99.764383,82,"$84,298.37"
3,apartment,Guerrero,16.829782,-99.911012,150,"$94,308.80"
4,house,Veracruz de Ignacio de la Llave,NaN,NaN,175,"$94,835.67"


It looks like there are a couple of problems in this DataFrame that you need to solve. First, there are many rows with `NaN` values in the `"lat"` and `"lon"` columns. Second, the data type for the `"price_usd"` column is `object` when it should be `float`. 

## Clean `df1`

**Task 1.2.3:** Clean `df1` by dropping rows with `NaN` values. Then remove the `"$"` and `","` characters from `"price_usd"` and recast the values in the column as floats.

In [6]:
# remove 'NaN' values
df1.dropna(inplace=True)

By default, pandas will keep the original DataFrame, and will create a copy that reflects the changes we just made. That's perfectly fine, but if we want to make sure that copies of the DataFrame aren't clogging up the memory on our computers, then we need to intervene with the `inplace` argument. `inplace=True` means that we want the original DataFrame updated without making a copy. If we don't include `inplace=True` (or if we do include `inplace=False`), then pandas will revert to the default. 

In [7]:
# transform price from object to float
df1["price_usd"]=(
    df1["price_usd"]
    .str.replace("$","",regex=False)
    .str.replace(",","")
    .astype(float)
)

In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 583 entries, 0 to 699
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   property_type  583 non-null    object 
 1   state          583 non-null    object 
 2   lat            583 non-null    float64
 3   lon            583 non-null    float64
 4   area_m2        583 non-null    int64  
 5   price_usd      583 non-null    float64
dtypes: float64(3), int64(1), object(2)
memory usage: 31.9+ KB


## Clean `df2`

Now it's time to tackle `df2`. Take a moment to inspect it using the same commands you used before. You'll notice that it has the same issue of `NaN` values, but there's a new problem, too: The home prices are in Mexican pesos (`"price_mxn"`), not US dollars (`"price_usd"`). If we want to compare all the home prices in this dataset, they all need to be in the same currency.

**Task 1.2.4:** First, drop rows with `NaN` values in `df2`. Next, use the `"price_mxn"` column to create a new column named `"price_usd"`. (Keep in mind that, when this data was collected in 2014, a dollar cost 19 pesos.) Finally, drop the `"price_mxn"` from the DataFrame.


In [9]:
# remove 'NaN' values
df2.dropna(inplace=True)

df2["price_usd"]=(df2["price_mxn"]/19).round(2)
df2.drop("price_mxn",axis=1,inplace=True)
df2.head()

,property_type,state,lat,lon,area_m2,price_usd
0,apartment,Nuevo León,25.721081,-100.345581,72,68421.05
2,house,Morelos,23.634501,-102.552788,360,278947.37
6,apartment,Estado de México,19.272040,-99.572013,85,65789.47
7,house,San Luis Potosí,22.138882,-100.996510,158,111578.95
8,apartment,Distrito Federal,19.394558,-99.129707,65,39904.74


## Clean `df3`

Great work! We're now on the final DataFrame. Use the same `shape`, `info` and `head` commands to inspect the `df3`. Do you see any familiar issues? 

You'll notice that we still have `NaN` values, but there are two new problems:

1. Instead of separate `"lat"` and `"lon"` columns, there's a single `"lat-lon"` column. 
2. Instead of a `"state"` column, there's a `"place_with_parent_names"` column.

We need the resolve these problems so that `df3` has the same columns in the same format as `df1` and `df2`.

**Task 1.2.5:** Drop rows with `NaN` values in `df3`. Then use the `split` method to create two new columns from `"lat-lon"` named `"lat"` and `"lon"`, respectively.

In [10]:
# remove 'NaN' values
df3.dropna(inplace=True)
# spliting lat-long col
df3[["lat","lon"]]=df3["lat-lon"].str.split(",",expand=True).astype(float)

Here, `expand` is telling pandas to make the DataFrame bigger; that is, to create a new column without dropping any of the ones that already exist.

In [11]:
df3.head()

,property_type,place_with_parent_names,lat-lon,area_m2,price_usd,lat,lon
0,apartment,|México|Distrito Federal|Gustavo A. Madero|Acu...,"19.52589,-99.151703",71,48550.59,19.525890,-99.151703
1,house,|México|Estado de México|Toluca|Metepec|,"19.2640539,-99.5727534",233,168636.73,19.264054,-99.572753
2,house,|México|Estado de México|Toluca|Toluca de Lerd...,"19.268629,-99.671722",300,86932.69,19.268629,-99.671722
4,apartment,|México|Veracruz de Ignacio de la Llave|Veracruz|,"19.511938,-96.871956",84,68508.67,19.511938,-96.871956
5,house,|México|Jalisco|Guadalajara|,"20.689157,-103.366728",175,102763.00,20.689157,-103.366728


**Task 1.2.6:** Use the [`split`](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.split.html) method again, this time to extract the state for every house. (Note that the state name always appears after `"México|"` in each string.) Use this information to create a `"state"` column. Finally, drop the `"place_with_parent_names"` and `"lat-lon"` columns from the DataFrame. 

In [12]:
df3["state"]=df3["place_with_parent_names"].str.split("|",expand=True)[2]
df3.drop(columns=["place_with_parent_names","lat-lon"],inplace=True)
new_col=["property_type","state","lat","lon","area_m2","price_usd"]
df3=df3.reindex(columns=new_col)
df3.head()

,property_type,state,lat,lon,area_m2,price_usd
0,apartment,Distrito Federal,19.525890,-99.151703,71,48550.59
1,house,Estado de México,19.264054,-99.572753,233,168636.73
2,house,Estado de México,19.268629,-99.671722,300,86932.69
4,apartment,Veracruz de Ignacio de la Llave,19.511938,-96.871956,84,68508.67
5,house,Jalisco,20.689157,-103.366728,175,102763.00


In [13]:
df3.head()

,property_type,state,lat,lon,area_m2,price_usd
0,apartment,Distrito Federal,19.525890,-99.151703,71,48550.59
1,house,Estado de México,19.264054,-99.572753,233,168636.73
2,house,Estado de México,19.268629,-99.671722,300,86932.69
4,apartment,Veracruz de Ignacio de la Llave,19.511938,-96.871956,84,68508.67
5,house,Jalisco,20.689157,-103.366728,175,102763.00


## Concatenate DataFrames

Great work! You have three clean DataFrames, and now it's time to combine them into a single DataFrame so that you can conduct your analysis.

**Task 1.2.7:** Use [`pd.concat`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html) to concatenate `df1`, `df2`, `df3` as new DataFrame named `df`. Your new DataFrame should have 1,736 rows and 6 columns:`"property_type"`, `"state"`, `"lat"`, `"lon"`, `"area_m2"`, `"price_usd"`, and `"price_per_m2"`. 

In [14]:
df = pd.concat([df1,df2,df3])
df.head()

,property_type,state,lat,lon,area_m2,price_usd
0,house,Estado de México,19.560181,-99.233528,150,67965.56
1,house,Nuevo León,25.688436,-100.198807,186,63223.78
2,apartment,Guerrero,16.767704,-99.764383,82,84298.37
3,apartment,Guerrero,16.829782,-99.911012,150,94308.80
5,house,Yucatán,21.052583,-89.538639,205,105191.37


In [15]:
df.shape

(1736, 6)

## Save `df`

The data is clean and in a single DataFrame, and now you need to save it as a CSV file so that you can examine it in your exploratory data analysis. 

**Task 1.2.8:** Save `df` as a CSV file using the [`to_csv`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html) method. The file path should be `"./data/mexico-real-estate-clean.csv"`. Be sure to set the `index` argument to `False`.


In [16]:
os.getcwd()

'C:\\Users\\Rw\\WorldQuant Project\\project_1'

Note that we're setting the `index` argument to `False` so that the DataFrame index isn't included in the CSV file.

In [17]:
df.to_csv("C:\\Users\\Rw\\WorldQuant Project\\project_1\\data\\mexico-real-estate-clean.csv",index=False)